<a href="https://colab.research.google.com/github/qqq3964/NLP-/blob/main/8%EC%9E%A5_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC(%EB%94%A5%EB%9F%AC%EB%8B%9D_%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98_%EA%B5%90%EA%B3%BC%EC%84%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 전처리

In [1]:
import nltk

nltk.download('punkt')
string1 = 'my favorite subject is math'
string2 = 'my favorite subject is math, english, economic and copmuter science'
nltk.word_tokenize(string1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['my', 'favorite', 'subject', 'is', 'math']

In [3]:
nltk.word_tokenize(string2) 

['my',
 'favorite',
 'subject',
 'is',
 'math',
 ',',
 'english',
 ',',
 'economic',
 'and',
 'copmuter',
 'science']

### konlpy 로 한국어 전처리

In [4]:
!sudo apt-get install g++ openjdk-11-jdk
!sudo apt-get install python3-dev; pip3 install konlpy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ is already the newest version (4:9.3.0-1ubuntu2).
g++ set to manually installed.
Suggested packages:
  openjdk-11-demo openjdk-11-source visualvm
The following NEW packages will be installed:
  openjdk-11-jdk
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 1,559 kB of archives.
After this operation, 1,624 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 openjdk-11-jdk amd64 11.0.18+10-0ubuntu1~20.04.1 [1,559 kB]
Fetched 1,559 kB in 0s (5,341 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
deb

In [5]:
from konlpy.tag import Komoran
komoran = Komoran()

In [6]:
print(komoran.morphs('딥러닝이 쉽나요? 어렵나요?'))

['딥러닝이', '쉽', '나요', '?', '어렵', '나요', '?']


In [7]:
# 품사태깅
print(komoran.pos('소파 위에 있는 것이 고양이인가요? 강아지인가요?'))

[('소파', 'NNP'), ('위', 'NNG'), ('에', 'JKB'), ('있', 'VV'), ('는', 'ETM'), ('것', 'NNB'), ('이', 'JKS'), ('고양이', 'NNG'), ('이', 'VCP'), ('ㄴ가요', 'EF'), ('?', 'SF'), ('강아지', 'NNG'), ('이', 'VCP'), ('ㄴ가요', 'EF'), ('?', 'SF')]


In [8]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 18.79 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Updating files: 100% (14737/14737), done.


In [20]:
import csv
from konlpy.tag import Okt
from gensim.models import word2vec

f = open('/content/nsmc/ratings_train.txt','r',encoding='utf-8')
rdr = csv.reader(f,delimiter='\t')
rdw = list(rdr)
f.close()

In [31]:
# 오픈소스 한글 형태소 분석기 호출
from tqdm import tqdm
twitter = Okt()

result = []

for line in tqdm(rdw):
  malist = twitter.pos(line[1],norm=True,stem=True)
  r = []
  for word in malist:
    # stopwords 제거 word[0] is word, word[1] is 품사 
    if not word[1] in ["Josa","Eomi","Punctuation"]:
      r.append(word[0])
  r1 = (" ".join(r)).strip()
  result.append(r1)

100%|██████████| 150001/150001 [13:07<00:00, 190.46it/s]


In [36]:
# result는 불용어를 제외한 나머지 의미있는 단어들을 ' ' 기준으로 만들어 놓았다.
print(result[:5])

['document', '아 더빙 진짜 짜증나다 목소리', '흠 포스터 보고 초딩 영화 줄 오버 연기 가볍다 않다', '너 무재 밓었 다그 래서 보다 추천 다', '교도소 이야기 구먼 솔직하다 재미 없다 평점 조정']


In [32]:
with open('NaverMovie.nlp','w',encoding='utf-8') as fp:
  fp.write('\n'.join(result))

In [50]:
from gensim.models import Word2Vec
'''
LineSentence은 gensim 패키지에서 제공하는 클래스 중 하나로, 
텍스트 파일을 한 줄씩 읽어서 각 줄에 있는 단어들을 리스트로 반환해주는 역할을 합니다.
'''
mData = word2vec.LineSentence("NaverMovie.nlp")
mModel = word2vec.Word2Vec(sentences=mData,size=100,window=10,hs=1,min_count=2,sg=1)
mModel.save('NaverMovie.model')

In [51]:
model = Word2Vec.load("/content/NaverMovie.model")

In [52]:
model.wv.most_similar('더빙')

[('성우', 0.8672624826431274),
 ('연예인', 0.7933328151702881),
 ('유행어', 0.7667208313941956),
 ('문성우', 0.7538083791732788),
 ('우리말', 0.7396552562713623),
 ('오딘', 0.7246542572975159),
 ('김원효', 0.7220051288604736),
 ('개그맨', 0.7164551019668579),
 ('메리다', 0.7122582793235779),
 ('최효종', 0.7108890414237976)]